# A. OpenNMT-py

- https://github.com/OpenNMT/OpenNMT-py

- 기본적으로 제공하는 데이터 및 task는 English to Germany 의 translation task

- summarization task로 적용하기 위해서 cnndm 뉴스 데이터셋을 적용


##  1.Setting

In [2]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git

Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 14846 (delta 14), reused 15 (delta 8), pack-reused 14809
Receiving objects: 100% (14846/14846), 146.21 MiB | 31.54 MiB/s, done.
Resolving deltas: 100% (10627/10627), done.


sent-summary data 다운로드

- https://github.com/harvardnlp/sent-summary

In [3]:
!ls

OpenNMT-py  sample_data


In [49]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd OpenNMT-py/

/content/OpenNMT-py


- http://opennmt.net/OpenNMT-py/Summarization.html

## 2.Process

### step1. Preprocess the data

In [6]:
!python preprocess.py -train_src data/src-train.txt -train_tgt data/tgt-train.txt -valid_src data/src-val.txt -valid_tgt data/tgt-val.txt -save_data data/demo

[2019-10-17 03:58:02,528 INFO] Extracting features...
[2019-10-17 03:58:02,529 INFO]  * number of source features: 0.
[2019-10-17 03:58:02,529 INFO]  * number of target features: 0.
[2019-10-17 03:58:02,529 INFO] Building `Fields` object...
[2019-10-17 03:58:02,529 INFO] Building & saving training data...
[2019-10-17 03:58:02,558 INFO] Building shard 0.
[2019-10-17 03:58:02,944 INFO]  * saving 0th train data shard to data/demo.train.0.pt.
[2019-10-17 03:58:03,621 INFO]  * tgt vocab size: 35820.
[2019-10-17 03:58:03,661 INFO]  * src vocab size: 24997.
[2019-10-17 03:58:03,831 INFO] Building & saving validation data...
[2019-10-17 03:58:03,933 INFO] Building shard 0.
[2019-10-17 03:58:04,036 INFO]  * saving 0th valid data shard to data/demo.valid.0.pt.


making demo files in the data folder

### Step2. Train the model

In [7]:
!nvidia-smi

Thu Oct 17 03:58:14 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!export CUDA_VISIBLE_DEVICES=0

In [0]:
# train on only cpu
# !python train.py -data data/demo -save_model demo-model

In [12]:
# train on single gpu
!python train.py -data data/demo -save_model demo-model -world_size 1 -gpu_ranks 0

[2019-10-16 09:50:40,112 INFO]  * src vocab size = 24997
[2019-10-16 09:50:40,113 INFO]  * tgt vocab size = 35820
[2019-10-16 09:50:40,113 INFO] Building model...
[2019-10-16 09:50:46,619 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(24997, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 500, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(35820, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3, inplace=False)
      (layers): ModuleList(
        (0): LSTMCell(1000, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=500, out_features=500, bias=False)
     

In [0]:
# model check point 드라이브에 저장
!cp -b 'OpenNMT-py/demo-model_step_5000.pt' 'drive/Shared drives/Hanseok/Colab Notebooks/NLP/Text_Summarization'

### Step3. Translate

In [10]:
!ls '../drive/Shared drives/Hanseok/Colab Notebooks/NLP/Text_Summarization/'

bottom-up-summary.ipynb   demo-model_step_10000.pt~  demo-model_step_25000.pt
data			  demo-model_step_15000.pt   demo-model_step_30000.pt
demo-model_step_10000.pt  demo-model_step_20000.pt   demo-model_step_5000.pt


In [12]:
!python translate.py -model '../drive/Shared drives/Hanseok/Colab Notebooks/NLP/Text_Summarization/demo-model_step_30000.pt' -src data/src-test.txt -output pred.txt -replace_unk -verbose

[2019-10-17 04:02:07,607 INFO] Translating shard 0.

SENT 1: ['Orlando', 'Bloom', 'and', 'Miranda', 'Kerr', 'still', 'love', 'each', 'other']
PRED 1: der Lesung: FACULTAD TV und der Nachbarort Anwednung anzeigen .
PRED SCORE: -5.8659

SENT 2: ['Actors', 'Orlando', 'Bloom', 'and', 'Model', 'Miranda', 'Kerr', 'want', 'to', 'go', 'their', 'separate', 'ways', '.']
PRED 2: Dabei lebt 5 Prozent und den Anfängen im Jahr 2000 .
PRED SCORE: -7.5029

SENT 3: ['However', ',', 'in', 'an', 'interview', ',', 'Bloom', 'has', 'said', 'that', 'he', 'and', 'Kerr', 'still', 'love', 'each', 'other', '.']
PRED 3: Er ist jedoch eine lange Übergangsphase , dass er eigentlich seine Ruhemöglichkeiten gibt .
PRED SCORE: -7.6046

SENT 4: ['Miranda', 'Kerr', 'and', 'Orlando', 'Bloom', 'are', 'parents', 'to', 'two-year-old', 'Flynn', '.']
PRED 4: Ursprünglich die Diffie-Hellman Gruppennummer und Rüge Nr.1 .
PRED SCORE: -4.9630

SENT 5: ['Actor', 'Orlando', 'Bloom', 'announced', 'his', 'separation', 'from', 'his', 

# B.CNN-DM 데이터셋 적용

- http://opennmt.net/OpenNMT-py/Summarization.html

- 기존의 ENG -> GER translation task 모델을 CNN_DM 뉴스 데이터셋으로 학습시켜서 summarization task 모델로 적용한다.

In [0]:
@inproceedings{gehrmann2018bottom,
  title={Bottom-Up Abstractive Summarization},
  author={Gehrmann, Sebastian and Deng, Yuntian and Rush, Alexander},
  booktitle={Proceedings of the 2018 Conference on Empirical Methods in Natural Language Processing},
  pages={4098--4109},
  year={2018}
}

In [45]:
!ls '../drive/Shared drives/Hanseok/Colab Notebooks/NLP/Text_Summarization/data/'

ls: cannot access '../drive/Shared drives/Hanseok/Colab Notebooks/NLP/Text_Summarization/data/': No such file or directory


In [52]:
!tar -zxvf '../drive/Shared drives/Hanseok/Colab Notebooks/NLP/Text_Summarization/data/cnndm.tar.gz' 

test.txt.src
test.txt.tgt.tagged
train.txt.src
train.txt.tgt.tagged
val.txt.src
val.txt.tgt.tagged


In [0]:
!mkdir data/cnndm

In [0]:
!mv val.txt.tgt.tagged data/cnndm/val.txt.tgt.tagged

In [0]:
!mv val.txt.tgt.tagged data/cnndm/val.txt.tgt.tagged
!mv val.txt.tgt.tagged data/cnndm/val.txt.tgt.tagged
!mv val.txt.tgt.tagged data/cnndm/val.txt.tgt.tagged
!mv val.txt.tgt.tagged data/cnndm/val.txt.tgt.tagged
!mv val.txt.tgt.tagged data/cnndm/val.txt.tgt.tagged
!mv val.txt.tgt.tagged data/cnndm/val.txt.tgt.tagged

## step1. Preprocess 

In [0]:
!python preprocess.py -train_src data/cnndm/train.txt.src \
                -train_tgt data/cnndm/train.txt.tgt.tagged \
                -valid_src data/cnndm/val.txt.src \
                -valid_tgt data/cnndm/val.txt.tgt.tagged \
                -save_data data/cnndm/CNNDM \
                -src_seq_length 10000 \
                -tgt_seq_length 10000 \
                -src_seq_length_trunc 400 \
                -tgt_seq_length_trunc 100 \
                -dynamic_dict \
                -share_vocab \
                -shard_size 100000


[2019-10-17 05:04:56,327 INFO] Extracting features...
[2019-10-17 05:04:56,327 INFO]  * number of source features: 0.
[2019-10-17 05:04:56,327 INFO]  * number of target features: 0.
[2019-10-17 05:04:56,328 INFO] Building `Fields` object...
[2019-10-17 05:04:56,328 INFO] Building & saving training data...
[2019-10-17 05:04:59,218 INFO] Building shard 0.
[2019-10-17 05:06:16,031 INFO]  * saving 0th train data shard to data/cnndm/CNNDM.train.0.pt.
tcmalloc: large alloc 1073741824 bytes == 0x1b355a000 @  0x7fab40f541e7 0x5bd1cb 0x4bd5d6 0x4be8a0 0x4bee44 0x4bf8b6 0x4bf46e 0x4c0335 0x4bf091 0x4beeb4 0x4bf51a 0x4c0335 0x4bf091 0x527d26 0x60c779 0x4f862c 0x4f98c7 0x4f6128 0x4f7d60 0x4f876d 0x4f98c7 0x4f6128 0x4f7d60 0x4f876d 0x4f98c7 0x4f7a28 0x4f876d 0x4f98c7 0x4f6128 0x4f7d60 0x4f876d
[2019-10-17 05:07:25,941 INFO] Building shard 1.
[2019-10-17 05:08:43,754 INFO]  * saving 1th train data shard to data/cnndm/CNNDM.train.1.pt.
[2019-10-17 05:09:39,734 INFO] Building shard 2.
[2019-10-17 05:1

## step2. Train

### 2-1) Base Model

In [0]:
!python train.py -save_model models/cnndm \
           -data data/cnndm/CNNDM \
           -copy_attn \
           -global_attention mlp \
           -word_vec_size 128 \
           -rnn_size 512 \
           -layers 1 \
           -encoder_type brnn \
           -train_steps 200000 \
           -max_grad_norm 2 \
           -dropout 0. \
           -batch_size 16 \
           -valid_batch_size 16 \
           -optim adagrad \
           -learning_rate 0.15 \
           -adagrad_accumulator_init 0.1 \
           -reuse_copy_attn \
           -copy_loss_by_seqlength \
           -bridge \
           -seed 777 \
           -world_size 1 \
           -gpu_ranks 0 

### 2-2) CNN-DM Transformer

In [0]:
!python train.py -data data/cnndm/CNNDM_Transformer \
           -save_model models/cnndm \
           -layers 4 \
           -rnn_size 512 \
           -word_vec_size 512 \
           -max_grad_norm 0 \
           -optim adam \
           -encoder_type transformer \
           -decoder_type transformer \
           -position_encoding \
           -dropout 0\.2 \
           -param_init 0 \
           -warmup_steps 8000 \
           -learning_rate 2 \
           -decay_method noam \
           -label_smoothing 0.1 \
           -adam_beta2 0.998 \
           -batch_size 4096 \
           -batch_type tokens \
           -normalization tokens \
           -max_generator_batches 2 \
           -train_steps 200000 \
           -accum_count 4 \
           -share_embeddings \
           -copy_attn \
           -param_init_glorot \
           -world_size 1 \
           -gpu_ranks 0 

## step3. Inference

In [0]:
!python translate.py -gpu X \
               -batch_size 20 \
               -beam_size 10 \
#                -model models/cnndm... \
               -src data/cnndm/test.txt.src \
               -output testout/cnndm.out \
               -min_length 35 \
               -verbose \
               -stepwise_penalty \
               -coverage_penalty summary \
               -beta 5 \
               -length_penalty wu \
               -alpha 0.9 \
               -verbose \
               -block_ngram_repeat 3 \
               -ignore_when_blocking "." "</t>" "<t>"

## step4. Evaluation

In [0]:
!python baseline.py -s testout/cnndm.out -t data/cnndm/test.txt.tgt.tagged -m sent_tag_verbatim -r